In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import gc
import math
import random
import time
import datetime
import boto3
import pickle

from io import StringIO
from collections import defaultdict
from numpy import argmax
from matplotlib import pyplot

from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

import sklearn.model_selection as model_selection
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import brier_score_loss
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.calibration import CalibratedClassifierCV

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.inspection import PartialDependenceDisplay
from sklearn.linear_model import LogisticRegression

import statsmodels.api as stm

from xgboost import XGBClassifier
import xgboost as xgb

import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
# check version number
import imblearn
print('The imblearn version is {}.'.format(imblearn.__version__))

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

#  Prep

In [ ]:
random.seed(10)

In [ ]:
data_dir = '/Users/Desktop/Classification/'

In [ ]:
US_STATES = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
US_REGIONS = ['MW', 'NE', 'NW', 'SE', 'SW', 'TX', 'UNKNOWN']
CUSTOMER_FACING = ['Y','N']
TRANSIT_OPERATORS =['SINGLE_DRIVER', 'TEAM_DRIVER']
DRIVER_TYPE = ['TEAM','SOLO1','SOLO2']
ACCOUNT_TYPE = ['outbound','transfer','empty','other'] 

In [ ]:
# read in train data up to and include week 49
loads = pd.read_csv(data_dir + 'train_49.csv',sep='\t', low_memory = False)
len(loads)

In [ ]:
loads.head()

In [ ]:
loads.creation_week_of_year.unique()

In [ ]:
loads = loads.drop('Unnamed: 0', axis=1)

In [ ]:
loads.head()

In [ ]:
loads.num_feasible_blocks.describe()

In [ ]:
loads[['load_id','creation_date']].sample(n=20, random_state=1)

In [ ]:
len(loads[loads.average_transit_hour > 690])

In [ ]:
list(loads.columns)

In [ ]:
len(loads)

In [ ]:
LANES = list(loads.lane.unique())

In [ ]:
loads.origin_zip3.head()

# Model Data Prep

In [ ]:
class_balancing = 'SMOTE'

MODEL_TYPE = 'RF'
MODEL_VALIDATION = True

WEEK_NUMBER = 50
YEAR = 2020

CUT_OFF_POINTS = [50,60,70] # for prediction of future weeks
WEEKEND_DAYS_TO_EXCLUDE = ['11/28/2021','11/29/2021','12/12/2021','12/13/2020']

In [ ]:
# Set Training and Testing Dates
training_start_date = '2020-06-15'
test_start_date_text = '2020-11-23'
test_end_date_text = '2020-12-02'
test_start_date_buffered = '2020-12-01'

In [ ]:
loads['origin_state'] = loads['origin_state'].astype('category')
loads['dest_state'] = loads['dest_state'].astype('category')
loads['lane'] = loads['lane'].astype('category')
loads['origin_zip3'] = loads['origin_zip3'].astype(int)
loads['dest_zip3'] = loads['dest_zip3'].astype(int)

In [ ]:
loads['departure_week_of_year'] = pd.to_datetime(loads['first_checkin_time_utc']).dt.weekofyear

In [ ]:
loads.info()

In [ ]:
loads['departure_hour_of_day'] = pd.to_datetime(loads['first_checkin_time_utc']).dt.hour
loads['departure_day_of_week'] = pd.to_datetime(loads['first_checkin_time_utc']).dt.dayofweek
loads['departure_week_of_year'] = pd.to_datetime(loads['first_checkin_time_utc']).dt.weekofyear
loads['departure_day_of_month'] = pd.to_datetime(loads['first_checkin_time_utc']).dt.day
loads['departure_month'] = pd.to_datetime(loads['first_checkin_time_utc']).dt.month

loads['sin_departure_time_hour_of_day'] = np.sin((2*np.pi*loads['departure_hour_of_day'])/24)
loads['cos_departure_time_hour_of_day'] = np.cos((2*np.pi*loads['departure_hour_of_day'])/24)

loads['creation_week_of_year'] = pd.to_datetime(loads['creation_date']).dt.weekofyear

In [ ]:
test_start_date_buffered = pd.to_datetime(test_start_date_buffered,format='%Y-%m-%d') 
test_start_date = pd.to_datetime(test_start_date_text,format='%Y-%m-%d') 
test_end_date = pd.to_datetime(test_end_date_text,format='%Y-%m-%d') 
train_start_date = pd.to_datetime(training_start_date,format='%Y-%m-%d') 


ml_test = loads[pd.to_datetime(loads.first_checkin_time_utc)>=test_start_date].copy()
ml_test = ml_test[pd.to_datetime(ml_test.first_checkin_time_utc)<test_end_date].copy()


X_test = ml_test.drop('is_eventually_unplanned',axis=1).copy()
y_test = ml_test['is_eventually_unplanned'].copy()


ml_train_val = loads[pd.to_datetime(loads.first_checkin_time_utc) >= train_start_date].copy()
ml_train_val = ml_train_val[pd.to_datetime(ml_train_val.first_checkin_time_utc) < test_start_date_buffered].copy()


X_train_val = ml_train_val.drop('is_eventually_unplanned',axis=1).copy()
y_train_val = ml_train_val['is_eventually_unplanned'].copy()


X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

print (X_train.shape)
print (y_train.shape)
print (X_val.shape)
print (y_val.shape)
print (X_test.shape)
print (y_test.shape)

# any common loads between data sets?
x = set(X_train.load_id.unique())
y = set(X_val.load_id.unique())
z = set(X_test.load_id.unique())

print(len(x.intersection(y)))
print(len(x.intersection(z)))
print(len(y.intersection(z)))

In [ ]:
x_train_departure_weeks = set(X_train.departure_week_of_year.unique())

In [ ]:
x_test_departure_weeks = set(X_test.departure_week_of_year.unique())

In [ ]:
test_unplanned_percent = (y_test.value_counts())[1] / len(y_test)
val_unplanned_percent = (y_val.value_counts())[1] / len(y_val)
train_unplanned_percent = (y_train.value_counts())[1] / len(y_train)

print(test_unplanned_percent)
print(val_unplanned_percent)
print(train_unplanned_percent)

In [ ]:
print (X_train.shape)
print (y_train.shape)
print (X_val.shape)
print (y_val.shape)
print (X_test.shape)
print (y_test.shape)

# Model Label Enconding

In [ ]:
sle = preprocessing.LabelEncoder()
sle.fit(US_STATES)

X_train['origin_state'] = sle.transform(X_train['origin_state'])
X_test['origin_state'] = sle.transform(X_test['origin_state'])
X_val['origin_state'] = sle.transform(X_val['origin_state'])

X_train['dest_state'] = sle.transform(X_train['dest_state'])
X_test['dest_state'] = sle.transform(X_test['dest_state'])
X_val['dest_state'] = sle.transform(X_val['dest_state'])

In [ ]:
features = [         
             'sin_departure_time_hour_of_day', # sin in reference to first checkin time
             'cos_departure_time_hour_of_day', # cos in reference to first checkin time
             'departure_hour_of_day', # untransformed hour in the day
             'departure_day_of_week',
             'departure_week_of_year', 
             'hook_trailer_min',
             'drop_trailer_min',
             'average_transit_hour',
             'miles',
             'checkin_time_windows_at_origin', # pickup window width
             'total_block_minutes',
             'num_feasible_blocks',
             'origin_zip3',
             'dest_zip3',
             'lead_time_to_departure'
           ]


load_related_features = ['load_id',
#                         'lane',
#                         'planning_status_by_blocks',
#                         'load_cancellation_date',
#                         'first_checkin_time_utc'
                        ]


In [ ]:
loads.head()

In [ ]:
loads.info()

In [ ]:
X_train_load_realted = X_train[load_related_features].copy()
X_test_load_related = X_test[load_related_features].copy()
X_val_load_related = X_val[load_related_features].copy()

In [ ]:
X_test_load_related.head()

In [ ]:
X_train = X_train[features].copy()
X_test = X_test[features].copy()
X_val = X_val[features].copy()

In [ ]:
X_test.head()

In [ ]:
print('check for null data in training set')
a = X_train[X_train.isnull().any(axis=1)] 
a.head()

In [ ]:
print('check for null data in test set')
b = X_test[X_test.isnull().any(axis=1)] 
b.head()

In [ ]:
print('check for null data in validation set')
c = X_val[X_val.isnull().any(axis=1)] 
c.head()

In [ ]:
X_train.info()


# Oversample and Undersample
#### This part plots the current class without over and under sampling - FOR PLOTS IN PAPER ONLY

In [ ]:
ys = y_train.to_numpy()
# using miles and average transit hours to demonstrate over and under sampling.
Xs = X_train[['miles','average_transit_hour']].to_numpy() 

In [ ]:
from collections import Counter
from numpy import where
counter = Counter(ys)
print(counter)

plt.rcParams["font.family"] = "Helvetica" # change matplotlib font family to Helvetica
# plt.rcParams['font.size'] = 12

csfont = {'fontname':'Helvetica'} # title font
hfont = {'fontname':'Helvetica'} # label font

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(ys == label)[0]
	pyplot.scatter(Xs[row_ix, 0], Xs[row_ix, 1], label=str(label))
    
pyplot.legend()
pyplot.title('Current Class Distribution Before Sampling', **csfont)
pyplot.savefig('SamplingCurrent.pdf')

#### Break down different over sampling and under sampling technique - FOR PLOTS IN PAPER ONLY

In [ ]:
X_train['sin_departure_time_hour_of_day'] = X_train['sin_departure_time_hour_of_day'].astype(int)
X_train['cos_departure_time_hour_of_day'] = X_train['cos_departure_time_hour_of_day'].astype(int)
X_train['miles'] = X_train['miles'].astype(int)
X_train['lead_time_to_departure'] = X_train['lead_time_to_departure'].astype(int)

In [ ]:
# SMOTE
sm = SMOTE(random_state=0) 
X_train_st, y_train_st = sm.fit_resample(X_train, y_train)
print('Shape of training data before over-sampling:')
print (X_train.shape)
print('Shape of training data after over-sampling:')
print (X_train_st.shape)
print('Accepted/ not accepted value counts before over-sampling:')
print (y_train.value_counts())
print('Accepted/ not accepted value counts after over-sampling:')
print (pd.Series(y_train_st).value_counts())

In [ ]:
# SMOTENN
smote_enn = SMOTEENN(random_state=0)
X_train_stenn, y_train_stenn = smote_enn.fit_resample(X_train, y_train)
print('Shape of training data before over-sampling:')
print (X_train.shape)
print('Shape of training data after over-sampling:')
print (X_train_stenn.shape)
print('Accepted/ not accepted value counts before over-sampling:')
print (y_train.value_counts())
print('Accepted/ not accepted value counts after over-sampling:')
print (pd.Series(y_train_stenn).value_counts())

In [ ]:
# SMOTETomek 
smote_tomek = SMOTETomek(random_state=0)
X_train_sttk, y_train_sttk  = smote_tomek.fit_resample(X_train, y_train)

print('Shape of training data before over-sampling:')
print (X_train.shape)
print('Shape of training data after over-sampling:')
print (X_train_sttk.shape)
print('Accepted/ not accepted value counts before over-sampling:')
print (y_train.value_counts())
print('Accepted/ not accepted value counts after over-sampling:')
print (pd.Series(y_train_sttk).value_counts())

In [ ]:
# near miss undersampling
nm1 = NearMiss(version=3)
X_train_nml, y_train_nml = nm1.fit_resample(X_train, y_train)
print('Shape of training data before over-sampling:')
print (X_train.shape)
print('Shape of training data after over-sampling:')
print (X_train_nml.shape)
print('Accepted/ not accepted value counts before over-sampling:')
print (y_train.value_counts())
print('Accepted/ not accepted value counts after over-sampling:')
print (pd.Series(y_train_nml).value_counts())

In [ ]:
plt.figure(figsize = (16,10))

# SMOTE Plot
plt.subplot(2,2,1)
y_st = y_train_st.to_numpy()
X_st = X_train_st[['miles','average_transit_hour']].to_numpy()

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_st == label)[0]
	pyplot.scatter(X_st[row_ix, 0], X_st[row_ix, 1], label=str(label))
    
pyplot.legend()
pyplot.title('SMOTE',**csfont)


# SMOTENN graph 
plt.subplot(2,2,2)
y_stenn = y_train_stenn.to_numpy()
X_stenn = X_train_stenn[['miles','average_transit_hour']].to_numpy()

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_stenn == label)[0]
	pyplot.scatter(X_stenn[row_ix, 0], X_stenn[row_ix, 1], label=str(label))
    
pyplot.legend()
pyplot.title('SMOTENN',**csfont)


# SMOTETomek graph 
plt.subplot(2,2,3)
y_sttk = y_train_sttk.to_numpy()
X_sttk = X_train_sttk[['miles','average_transit_hour']].to_numpy()

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_sttk == label)[0]
	pyplot.scatter(X_sttk[row_ix, 0], X_sttk[row_ix, 1], label=str(label))
    
pyplot.legend()
pyplot.title('SMOTETomek',**csfont)


# near miss plot
plt.subplot(2,2,4)
y_nml = y_train_nml.to_numpy()
X_nml = X_train_nml[['miles','average_transit_hour']].to_numpy()

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_nml == label)[0]
	pyplot.scatter(X_nml[row_ix, 0], X_nml[row_ix, 1], label=str(label))
    
pyplot.legend()
pyplot.title('Near Miss',**csfont)

plt.savefig('Sampling4Plot.pdf')


### Loop the four sampling techniques together - This is used for the actual model building process

In [ ]:
# Combining over and under sampling 
# Tomek's link
if class_balancing == 'SMOTENN':
    smote_enn = SMOTEENN(random_state=0)
    X_train_res, y_train_res = smote_enn.fit_resample(X_train, y_train)

    print('Shape of training data before over-sampling:')
    print (X_train.shape)
    print('Shape of training data after over-sampling:')
    print (X_train_res.shape)
    print('Accepted/ not accepted value counts before over-sampling:')
    print (y_train.value_counts())
    print('Accepted/ not accepted value counts after over-sampling:')
    print (pd.Series(y_train_res).value_counts())

elif class_balancing =='Near Miss':

#Balance classes by undersampling - Near Miss  
    nm1 = NearMiss(version=3)
    X_train_res, y_train_res = nm1.fit_resample(X_train, y_train)
    print('Shape of training data before over-sampling:')
    print (X_train.shape)
    print('Shape of training data after over-sampling:')
    print (X_train_res.shape)
    print('Accepted/ not accepted value counts before over-sampling:')
    print (y_train.value_counts())
    print('Accepted/ not accepted value counts after over-sampling:')
    print (pd.Series(y_train_res).value_counts())

elif class_balancing == 'SMOTETomek':
    smote_tomek = SMOTETomek(random_state=0)
    X_train_res, y_train_res  = smote_tomek.fit_resample(X_train, y_train)

    print('Shape of training data before over-sampling:')
    print (X_train.shape)
    print('Shape of training data after over-sampling:')
    print (X_train_res.shape)
    print('Accepted/ not accepted value counts before over-sampling:')
    print (y_train.value_counts())
    print('Accepted/ not accepted value counts after over-sampling:')
    print (pd.Series(y_train_res).value_counts())

elif class_balancing == 'SMOTE':
    sm = SMOTE(random_state=0) 
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
    print('Shape of training data before over-sampling:')
    print (X_train.shape)
    print('Shape of training data after over-sampling:')
    print (X_train_res.shape)
    print('Accepted/ not accepted value counts before over-sampling:')
    print (y_train.value_counts())
    print('Accepted/ not accepted value counts after over-sampling:')
    print (pd.Series(y_train_res).value_counts())

else:
    #assert('Missing class balancing')
    X_train_res = X_train.copy()
    y_train_res = y_train.copy()

In [ ]:
X_train_res.info()

In [ ]:
(pd.Series(y_val).value_counts()) 

In [ ]:
(pd.Series(y_test).value_counts()) 

In [ ]:
(pd.Series(y_train).value_counts()) 

In [ ]:
(pd.Series(y_train_res).value_counts()) 

# HyperParameter Tuning and Grid Search
### Takes a VERY VERY long time to run! 

In [ ]:
# Random Forest 
rf_param_grid = {
            'max_depth': [None,10,20],
            'max_features' : ['sqrt'], 
            'min_samples_leaf': [15,25,50],
            'n_estimators': [250,500,1000],
            'criterion' : ['entropy'],
            'oob_score': [True],
            'bootstrap': [True]
        }

rf_model = RandomForestClassifier(random_state = 0)
rf_grid_search = GridSearchCV(estimator = rf_model, param_grid = rf_param_grid, cv = 3, n_jobs = 10,
                                      verbose = 1,scoring ='neg_brier_score') # used brier score for tunning

rf_grid_search.fit(X_train_res, y_train_res)

rf_best_params = rf_grid_search.best_params_
rf_best_grid_search_score = rf_grid_search.best_score_
rf_best_model = rf_grid_search.best_estimator_
        
pickle.dump(rf_best_params,open('rf_best_params.p', 'wb'))

In [ ]:
rf_best_params 

In [ ]:
rf_best_grid_search_score

In [ ]:
rf_best_model

In [ ]:
# XGB 
xgb_param_grid = {
            'eta': [0.01,0.05, 0.3] ,
            'max_depth' : [3,6],
            'gamma' : [0,1,10],
            'min_child_weight': [3,6,9],
            'n_estimators': [250,500]
        }

xgb_model = XGBClassifier(random_state=0) 
xgb_grid_search = GridSearchCV(estimator = xgb_model, param_grid = xgb_param_grid, cv = 3, n_jobs = 10, 
                               verbose = 1,scoring ='neg_brier_score') # used brier score for tunning
xgb_grid_search.fit(X_train_res, y_train_res)

xgb_best_params = xgb_grid_search.best_params_
xgb_best_grid_search_score = xgb_grid_search.best_score_

    
pickle.dump(xgb_best_params,open('xgb_best_params.p', 'wb'))   

In [ ]:
xgb_best_params

In [ ]:
xgb_best_grid_search_score

In [ ]:
xgb_best_model = xgb_grid_search.best_estimator_
xgb_best_model

In [ ]:
stmmodel = stm.Probit(y_train_res, X_train_res)
pb = stmmodel.fit()
pb.summary()

In [ ]:
stmmodel = stm.Logit(y_train_res, X_train_res)
result = stmmodel.fit(method='newton')
result.summary()

In [ ]:
lr = LogisticRegression(random_state=0, max_iter = 5000,n_jobs = 10)
lr_result = lr.fit(X_train_res, y_train_res)

In [ ]:
print(lr_result)

In [ ]:
pd.DataFrame(lr_result.intercept_)

In [ ]:
pd.DataFrame(lr_result.coef_)

In [ ]:
lr_result.feature_names_in_

In [ ]:
lr_result.classes_

In [ ]:
lr.predict(X_train_res)

In [ ]:
lr.predict_proba(X_train_res)

In [ ]:
xgb_best_model.predict_proba(X_train_res)

In [ ]:
rf_best_model.predict_proba(X_train_res)

In [ ]:
X_test.info()

# Comparison of Calibration of Classifiers

In [ ]:
from sklearn.calibration import CalibrationDisplay
from matplotlib.gridspec import GridSpec

In [ ]:
# Create classifiers
clf_list = [
        (rf_best_model, "Random Forest Classifier"),
        (xgb_best_model, "XGB Classifier"),
        (lr, "Logistic Regression")
           ]

In [ ]:
fig = plt.figure(figsize=(10, 8))
colors = plt.cm.get_cmap("Dark2")
gs = GridSpec(3, 3)

          
calibration_displays = {}
ax_calibration_curve = fig.add_subplot(gs[:2, :2])
for i, (clf, name) in enumerate(clf_list):
    display = CalibrationDisplay.from_estimator(
        clf,
        X_test,
        y_test,
        n_bins=10,
        name=name,
        ax=ax_calibration_curve,
        color=colors(i),
    )
    calibration_displays[name] = display

# ax_calibration_curve.grid()
ax_calibration_curve.set_title("Model Calibration Plots")

plt.savefig('Cal_Classifier_Plots.pdf')

In [ ]:
fig = plt.figure(figsize=(11, 10))
colors = plt.cm.get_cmap("Dark2")
gs = GridSpec(3, 3)

grid_positions = [(0, 0), (0, 1), (0, 2)]
plt.tight_layout()
for i, (_, name) in enumerate(clf_list):
    row, col = grid_positions[i]
    ax = fig.add_subplot(gs[row, col])
    ax.hist(
        calibration_displays[name].y_prob,
        range=(0, 1),
        bins=10,
        label=name,
        color=colors(i),
    )
    ax.set(title=name, xlabel="Mean Predicted Probability", ylabel="Count")
plt.savefig('Cal_Classifier_Hist.pdf')

plt.show()

# Probability Calibration Curves

### Random Forest - Probability Calibration Curves

In [ ]:
# Create classifiers
rf_sigmoid = CalibratedClassifierCV(rf_best_model, method="sigmoid", cv="prefit")
        
# isotonic 
rf_isotonic = CalibratedClassifierCV(rf_best_model, method="isotonic", cv="prefit")

clf_list = [
        (rf_best_model, "Random Forest Classifier"),
        (rf_sigmoid, "Random Forest + Sigmoid"),
        (rf_isotonic, "Random Forest + Isotonic")
           ]

In [ ]:
fig = plt.figure(figsize=(10, 8))
colors = plt.cm.get_cmap("Dark2_r")
          
calibration_displays = {}
ax_calibration_curve = fig.add_subplot(gs[:2, :2])
for i, (clf, name) in enumerate(clf_list):
    clf.fit(X_train_res, y_train_res)
    display = CalibrationDisplay.from_estimator(
        clf,
        X_test,
        y_test,
        n_bins=10,
        name=name,
        ax=ax_calibration_curve,
        color=colors(i),
    )
    calibration_displays[name] = display

# ax_calibration_curve.grid()
ax_calibration_curve.set_title("Probability Calibration Plots - Random Forest")

plt.savefig('Cal_Prob_RF.pdf')

In [ ]:
fig = plt.figure(figsize=(11, 10))
colors = plt.cm.get_cmap("Dark2_r")
gs = GridSpec(3, 3)

grid_positions = [(0, 0), (0, 1), (0, 2)]
plt.tight_layout()
for i, (_, name) in enumerate(clf_list):
    row, col = grid_positions[i]
    ax = fig.add_subplot(gs[row, col])
    ax.hist(
        calibration_displays[name].y_prob,
        range=(0, 1),
        bins=10,
        label=name,
        color= colors(i),
    )
    ax.set(title=name, xlabel="Mean Predicted Probability", ylabel="Count")
plt.savefig('Cal_Pro_Hist_RF.pdf')

plt.show()

### XGB -  Probability Calibration Curves

In [ ]:
# Create classifiers
xgb_sigmoid = CalibratedClassifierCV(xgb_best_model, method="sigmoid", cv="prefit")
        
# isotonic 
xgb_isotonic = CalibratedClassifierCV(xgb_best_model, method="isotonic", cv="prefit")

clf_list = [
        (xgb_best_model, "XGB Classifier"),
        (xgb_sigmoid, "XGB + Sigmoid"),
        (xgb_isotonic, "XGB + Isotonic")
           ]

In [ ]:
fig = plt.figure(figsize=(10, 8))
colors = plt.cm.get_cmap("Dark2_r")
          
calibration_displays = {}
ax_calibration_curve = fig.add_subplot(gs[:2, :2])
for i, (clf, name) in enumerate(clf_list):
    clf.fit(X_train_res, y_train_res)
    display = CalibrationDisplay.from_estimator(
        clf,
        X_test,
        y_test,
        n_bins=10,
        name=name,
        ax=ax_calibration_curve,
        color=colors(i),
    )
    calibration_displays[name] = display

# ax_calibration_curve.grid()
ax_calibration_curve.set_title("Probability Calibration Plots - XGB")

plt.savefig('Cal_Prob_XGB.pdf')

In [ ]:
fig = plt.figure(figsize=(11, 10))
colors = plt.cm.get_cmap("Dark2_r")
gs = GridSpec(3, 3)

grid_positions = [(0, 0), (0, 1), (0, 2)]
plt.tight_layout()
for i, (_, name) in enumerate(clf_list):
    row, col = grid_positions[i]
    ax = fig.add_subplot(gs[row, col])
    ax.hist(
        calibration_displays[name].y_prob,
        range=(0, 1),
        bins=10,
        label=name,
        color= colors(i),
    )
    ax.set(title=name, xlabel="Mean Predicted Probability", ylabel="Count")
plt.savefig('Cal_Pro_Hist_XGB.pdf')

plt.show()

### Logistic Regression - Probability Calibration Curves

In [ ]:
# Create classifiers
lr_sigmoid = CalibratedClassifierCV(lr, method="sigmoid", cv="prefit")
        
# isotonic 
lr_isotonic = CalibratedClassifierCV(lr, method="isotonic", cv="prefit")

clf_list = [
        (lr, "Logistic Regression"),
        (lr_sigmoid, "Logistic Regression + Sigmoid"),
        (lr_isotonic, "Logistic Regression + Isotonic")
           ]

In [ ]:
fig = plt.figure(figsize=(10, 8))
colors = plt.cm.get_cmap("Dark2_r")
          
calibration_displays = {}
ax_calibration_curve = fig.add_subplot(gs[:2, :2])
for i, (clf, name) in enumerate(clf_list):
    clf.fit(X_train_res, y_train_res)
    display = CalibrationDisplay.from_estimator(
        clf,
        X_test,
        y_test,
        n_bins=10,
        name=name,
        ax=ax_calibration_curve,
        color=colors(i),
    )
    calibration_displays[name] = display

# ax_calibration_curve.grid()
ax_calibration_curve.set_title("Probability Calibration Plots - Logistic Regression")

plt.savefig('Cal_Prob_LR.pdf')

In [ ]:
fig = plt.figure(figsize=(11, 10))
colors = plt.cm.get_cmap("Dark2_r")
gs = GridSpec(3, 3)

grid_positions = [(0, 0), (0, 1), (0, 2)]
plt.tight_layout()
for i, (_, name) in enumerate(clf_list):
    row, col = grid_positions[i]
    ax = fig.add_subplot(gs[row, col])
    ax.hist(
        calibration_displays[name].y_prob,
        range=(0, 1),
        bins=10,
        label=name,
        color= colors(i),
    )
    ax.set(title=name, xlabel="Mean Predicted Probability", ylabel="Count")
plt.savefig('Cal_Pro_Hist_LR.pdf')

plt.show()

# Looping to Select ML Models

In [ ]:
## RF HYPERPARAMETER TUNNING - GRID SEARCH
if MODEL_TYPE == 'RF' : 
    
    if MODEL_VALIDATION == True:

        rf_param_grid = {
            'max_depth': [None,10,20],
            'max_features' : ['sqrt'], 
            'min_samples_leaf': [15,25,50],
            'n_estimators': [250,500,1000],
            'criterion' : ['entropy'],
            'oob_score': [True],
            'bootstrap': [True]
        }

        rf_model = RandomForestClassifier(random_state = 0)
        rf_grid_search = GridSearchCV(estimator = rf_model, param_grid = rf_param_grid, cv = 3, n_jobs = 10,
                                      verbose = 1,scoring ='neg_brier_score') # used brier score for tunning
        rf_grid_search.fit(X_train_res, y_train_res)

        rf_best_params = rf_grid_search.best_params_
        rf_best_grid_search_score = rf_grid_search.best_score_
        rf_best_model = rf_grid_search.best_estimator_
        
        pickle.dump(rf_best_params,open('rf_best_params.p', 'wb'))
    
    
### XGBOOST - HYPERPARAMETER TUNNING - GRID SEARCH        

if MODEL_TYPE == 'XGB':
    
    if MODEL_VALIDATION == True :
        xgb_param_grid = {
            'eta': [0.01,0.05, 0.3] ,
            'max_depth' : [3,6],
            'gamma' : [0,1,10],
            'min_child_weight': [3,6,9],
            'n_estimators': [250,500]
        }

        xgb_model = XGBClassifier(random_state=0) 
        xgb_grid_search = GridSearchCV(estimator = xgb_model, param_grid = xgb_param_grid, cv = 3, n_jobs = 10, verbose = 1,scoring ='neg_brier_score') # used brier score for tunning
        xgb_grid_search.fit(X_train_res, y_train_res)

        xgb_best_params = xgb_grid_search.best_params_
        xgb_best_grid_search_score = xgb_grid_search.best_score_

    
        pickle.dump(xgb_best_params,open('xgb_best_params.p', 'wb'))
        
if MODEL_TYPE == 'LR': 
    lr = LogisticRegression(random_state=0)
    lr.fit(X_train_res, y_train_res)
    
# if MODEL_TYPE == 'STM':
#   model = stm.Logit(y_train_res, X_train_res)
#   result = model.fit(method='newton')
#   result.summary()   
#     logmodel=stm.GLM(y_train_res,stm.add_constant(X_train_res),family=stm.families.Binomial())
#     result = logmodel.fit()
    

In [ ]:
start_time = time.time()
if MODEL_TYPE == 'RF': 
#     rf_best_params = pickle.load(open('rf_best_params.p', 'rb'))
    
#     rf_best_model = RandomForestClassifier(n_estimators = rf_best_params['n_estimators'], random_state = 0,max_features = rf_best_params['max_features'],n_jobs=10,verbose=1,oob_score=True, min_samples_leaf = rf_best_params['min_samples_leaf'],  max_depth = rf_best_params['max_depth'] ,criterion= 'entropy' )
#     rf_best_model.fit(X_train_res,y_train_res)
    
    # tuned model
    rf_best_model = RandomForestClassifier(n_estimators = 1000, random_state = 0,max_features ='sqrt', n_jobs=10,
                                           verbose=1,oob_score=True, min_samples_leaf = 15,  
                                           max_depth = None ,bootstrap = True,criterion= 'entropy' )
    rf_best_model.fit(X_train_res,y_train_res)
    #rf_best_model.fit(X_train,y_train)
    
end_time = time.time()
elapsed_time = (end_time - start_time)
print(elapsed_time) 


In [ ]:
# XGB Boost
xgb_best_params = pickle.load(open('xgb_best_params.p', 'rb'))
train_x = X_train_res.values
train_y = y_train_res.values
test_x = X_test.values
val_x = X_val.values
val_y = y_val.values
test_y = y_test.values
xgb_best_model = XGBClassifier(n_estimators =xgb_best_params['n_estimators'],
                                   random_state=0,verbose=1,
                                   eta = xgb_best_params['eta'],
                                   max_depth = xgb_best_params['max_depth'], 
                                   min_child_weight = xgb_best_params['min_child_weight'],
                                   gamma = xgb_best_params['gamma'])
    

xgb_best_model.fit(train_x,train_y)

In [ ]:
len(X_test)

## Define function to get model fit statistics

In [ ]:
def get_metrics(y,y_hat,y_prob_hat):

    acc_score = round(accuracy_score(y, y_hat),4)
    brier_score = round(brier_score_loss(y,y_prob_hat[:,1], pos_label=1),4)
    log_loss_score = round(log_loss(y,y_prob_hat, labels=[0,1]),4)    
    fpr, tpr, thresholds = metrics.roc_curve(y,y_prob_hat[:,1])
    AUC = metrics.auc(fpr, tpr)     
    
    return acc_score,brier_score,log_loss_score,AUC 

def get_results_from_tunning_RF_LR(tuned_model):
        
    # Train data
    y_train_pred = tuned_model.predict(X_train_res)
    y_train_probs_uncallibrated = tuned_model.predict_proba(X_train_res)

    train_accuracy_score, train_brier_score, train_log_loss_score, train_AUC = get_metrics(y_train_res,y_train_pred,y_train_probs_uncallibrated)
    

    # Validation
    y_val_pred = tuned_model.predict(X_val)
    y_val_probs_uncallibrated = tuned_model.predict_proba(X_val)
    
    val_accuracy_score,val_brier_score,val_log_loss_score,val_AUC = get_metrics(y_val,y_val_pred,y_val_probs_uncallibrated)
    
    # Test data
    y_test_pred = tuned_model.predict(X_test)
    y_test_probs_uncallibrated = tuned_model.predict_proba(X_test)
    
    test_accuracy_score,test_brier_score,test_log_loss_score,test_AUC = get_metrics(y_test, y_test_pred, y_test_probs_uncallibrated)

  
    # Callibrated  - test data
    
    #sigmoid
    callibrated_sigmoid = CalibratedClassifierCV(tuned_model, method="sigmoid", cv="prefit") # isotonic and sigmoid is tested - isotonic performed better
    callibrated_sigmoid.fit(X_val, y_val)
    
    callibrated_sigmoid_probs = callibrated_sigmoid.predict_proba(X_test)
    y_test_pred_callibrated_sigmoid = callibrated_sigmoid.predict(X_test)
        
    sigmoid_accuracy_score,brier_score_sigmoid,callibrated_log_loss_sigmoid,sigmoid_AUC = get_metrics(y_test, y_test_pred_callibrated_sigmoid, callibrated_sigmoid_probs)

    # isotonic 
    callibrated_isotonic = CalibratedClassifierCV(tuned_model, method="isotonic", cv="prefit")
    callibrated_isotonic.fit(X_val,y_val)
    
    callibrated_isotonic_probs = callibrated_isotonic.predict_proba(X_test)
    y_test_pred_callibrated_isotonic = callibrated_isotonic.predict(X_test)
    
    isotonic_accuracy_score,brier_score_isotonic,callibrated_log_loss_isotonic,isotonic_AUC = get_metrics(y_test, y_test_pred_callibrated_isotonic, callibrated_isotonic_probs)
    
    
    results = [train_accuracy_score,
               train_brier_score,
               train_log_loss_score,
               train_AUC,
               
               val_accuracy_score,
               val_brier_score,
               val_log_loss_score,
               val_AUC,
               
               test_accuracy_score,
               test_brier_score,
               test_log_loss_score,
               test_AUC,
               
               sigmoid_accuracy_score,
               brier_score_sigmoid,
               callibrated_log_loss_sigmoid,
               sigmoid_AUC,
               
               isotonic_accuracy_score,
               brier_score_isotonic,
               callibrated_log_loss_isotonic,
               isotonic_AUC
    
    
    ]    
    return results, callibrated_isotonic


def get_results_from_tunning_XGB(tuned_model):
        
    # Train data
    y_train_pred = tuned_model.predict(train_x) 
    y_train_probs_uncallibrated = tuned_model.predict_proba(train_x)

    train_accuracy_score, train_brier_score, train_log_loss_score, train_AUC = get_metrics(train_y,y_train_pred,y_train_probs_uncallibrated)
    

    # Validation
    y_val_pred = tuned_model.predict(val_x) 
    y_val_probs_uncallibrated = tuned_model.predict_proba(val_x)
    
    val_accuracy_score,val_brier_score,val_log_loss_score,val_AUC = get_metrics(val_y,y_val_pred,y_val_probs_uncallibrated)
    
    # Test data
    y_test_pred = tuned_model.predict(test_x) 
    y_test_probs_uncallibrated = tuned_model.predict_proba(test_x)

    test_accuracy_score,test_brier_score,test_log_loss_score,test_AUC = get_metrics(test_y, y_test_pred, y_test_probs_uncallibrated)

  
    # Callibrated  - test data
    
    #sigmoid
    callibrated_sigmoid = CalibratedClassifierCV(tuned_model, method="sigmoid", cv="prefit") 
    callibrated_sigmoid.fit(val_x,val_y)
    
    callibrated_sigmoid_probs = callibrated_sigmoid.predict_proba(test_x)
    y_test_pred_callibrated_sigmoid = callibrated_sigmoid.predict(test_x)
        
    sigmoid_accuracy_score,brier_score_sigmoid,callibrated_log_loss_sigmoid,sigmoid_AUC = get_metrics(test_y, y_test_pred_callibrated_sigmoid, callibrated_sigmoid_probs)

    # isotonic 
    callibrated_isotonic = CalibratedClassifierCV(tuned_model, method="isotonic", cv="prefit")
    callibrated_isotonic.fit(val_x,val_y)
    
    callibrated_isotonic_probs = callibrated_isotonic.predict_proba(test_x)
    y_test_pred_callibrated_isotonic = callibrated_isotonic.predict(test_x)
    
    isotonic_accuracy_score,brier_score_isotonic,callibrated_log_loss_isotonic,isotonic_AUC = get_metrics(test_y, y_test_pred_callibrated_isotonic, callibrated_isotonic_probs)
    
    
    results = [train_accuracy_score,
               train_brier_score,
               train_log_loss_score,
               train_AUC,
               
               val_accuracy_score,
               val_brier_score,
               val_log_loss_score,
               val_AUC,
               
               test_accuracy_score,
               test_brier_score,
               test_log_loss_score,
               test_AUC,
               
               sigmoid_accuracy_score,
               brier_score_sigmoid,
               callibrated_log_loss_sigmoid,
               sigmoid_AUC,
               
               isotonic_accuracy_score,
               brier_score_isotonic,
               callibrated_log_loss_isotonic,
               isotonic_AUC      
    ]    
    return results,callibrated_isotonic

# Plot ROC Curve

In [ ]:
# Training Separate Curves 

from sklearn.metrics import roc_curve

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 3.4))
# fig.tight_layout() # Or equivalently,  "plt.tight_layout()"

plt.subplot(1,3,1)
fpr, tpr, thresholds = roc_curve(y_train_res, rf_best_model.predict_proba(X_train_res)[:,1])
auc = roc_auc_score(y_train_res, rf_best_model.predict(X_train_res))
plt.plot(fpr,tpr,label='Random Forest (area = %0.2f)' %auc, color= '#1b9e77')
plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Train - Random Forest Classifier')

plt.subplot(1,3,2)
fpr, tpr, thresholds = roc_curve(y_train_res, xgb_best_model.predict_proba(X_train_res)[:,1])
auc = roc_auc_score(y_train_res, xgb_best_model.predict(X_train_res))
plt.plot(fpr,tpr,label='XGB Classifier (area = %0.2f)' %auc, color ='#d95f02')
plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Train - XGB Classifier')

plt.subplot(1,3,3)
fpr, tpr, thresholds = roc_curve(y_train_res, lr.predict_proba(X_train_res)[:,1])
auc = roc_auc_score(y_train_res, lr.predict(X_train_res))
plt.plot(fpr,tpr,label='Logistics Regression (area = %0.2f)' %auc, color = '#7570b3')
plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Train - Logistic Regression')

plt.savefig('ROC_Train_Split.pdf')

In [ ]:
# Train All Together

from sklearn.metrics import roc_curve
plt.figure(figsize = (5,5))

fpr, tpr, thresholds = roc_curve(y_train_res, rf_best_model.predict_proba(X_train_res)[:,1])
auc = round(roc_auc_score(y_train_res, rf_best_model.predict(X_train_res)), 2)
plt.plot(fpr,tpr,label='Random Forest, AUC='+str(auc), color='#1b9e77')


fpr, tpr, thresholds = roc_curve(y_train_res, xgb_best_model.predict_proba(X_train_res)[:,1])
auc = round(roc_auc_score(y_train_res, xgb_best_model.predict(X_train_res)),2)
plt.plot(fpr,tpr,label='XGB Classifier, AUC='+str(auc), color ='#d95f02')


fpr, tpr, thresholds = roc_curve(y_train_res, lr.predict_proba(X_train_res)[:,1])
auc = round(roc_auc_score(y_train_res, lr.predict(X_train_res)),2)
plt.plot(fpr,tpr,label='Logistics Regression, AUC='+str(auc), color='#7570b3') 

plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Train Data')

plt.savefig('ROC_Train_All.pdf')

In [ ]:
X_val['departure_week_of_year'] = X_val['departure_week_of_year'].astype(int)
X_test['departure_week_of_year'] = X_test['departure_week_of_year'].astype(int)

In [ ]:
# Validation Separate Plots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 3.4))
# fig.tight_layout() # Or equivalently,  "plt.tight_layout()"

plt.subplot(1,3,1)
fpr, tpr, thresholds = roc_curve(y_val, rf_best_model.predict_proba(X_val)[:,1])
auc = roc_auc_score(y_val, rf_best_model.predict(X_val))
plt.plot(fpr,tpr,label='Random Forest (area = %0.2f)' %auc, color='#1b9e77')
plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Validation Data - Random Forest Classifier')

plt.subplot(1,3,2)
fpr, tpr, thresholds = roc_curve(y_val, xgb_best_model.predict_proba(X_val)[:,1])
auc = roc_auc_score(y_val, xgb_best_model.predict(X_val))
plt.plot(fpr,tpr,label='XGB Classifier (area = %0.2f)' %auc, color ='#d95f02')
plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Validation Data - XGB Classifier')

plt.subplot(1,3,3)
fpr, tpr, thresholds = roc_curve(y_val, lr.predict_proba(X_val)[:,1])
auc = roc_auc_score(y_val, lr.predict(X_val))
plt.plot(fpr,tpr,label='Logistics Regression (area = %0.2f)' %auc,color='#7570b3')
plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Validation Data - Logistic Regression')

plt.savefig('ROC_Val_Split.pdf')

In [ ]:
# Validation All Together
plt.figure(figsize = (5,5))

fpr, tpr, thresholds = roc_curve(y_val, rf_best_model.predict_proba(X_val)[:,1])
auc = round(roc_auc_score(y_val, rf_best_model.predict(X_val)), 2)
plt.plot(fpr,tpr,label='Random Forest, AUC='+str(auc), color='#1b9e77')


fpr, tpr, thresholds = roc_curve(y_val, xgb_best_model.predict_proba(X_val)[:,1])
auc = round(roc_auc_score(y_val, xgb_best_model.predict(X_val)),2)
plt.plot(fpr,tpr,label='XGB Classifier, AUC='+str(auc), color ='#d95f02')


fpr, tpr, thresholds = roc_curve(y_val, lr.predict_proba(X_val)[:,1])
auc = round(roc_auc_score(y_val, lr.predict(X_val)),2)
plt.plot(fpr,tpr,label='Logistics Regression, AUC='+str(auc), color='#7570b3') #7E1E9C

plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Validation Data')

plt.savefig('ROC_Val_All.pdf')

In [ ]:
# Test Seperate

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 3.4))
# fig.tight_layout() # Or equivalently,  "plt.tight_layout()"

plt.subplot(1,3,1)
fpr, tpr, thresholds = roc_curve(y_test, rf_best_model.predict_proba(X_test)[:,1])
auc = roc_auc_score(y_test, rf_best_model.predict(X_test))
plt.plot(fpr,tpr,label='Random Forest (area = %0.2f)' %auc, color='#1b9e77')
plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Test Data - Random Forest Classifier')

plt.subplot(1,3,2)
fpr, tpr, thresholds = roc_curve(y_test, xgb_best_model.predict_proba(X_test)[:,1])
auc = roc_auc_score(y_test, xgb_best_model.predict(X_test))
plt.plot(fpr,tpr,label='XGB Classifier (area = %0.2f)' %auc, color ='#d95f02')
plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Test Data - XGB Classifier')

plt.subplot(1,3,3)
fpr, tpr, thresholds = roc_curve(y_test, lr.predict_proba(X_test)[:,1])
auc = roc_auc_score(y_test, lr.predict(X_test))
plt.plot(fpr,tpr,label='Logistics Regression (area = %0.2f)' %auc, color='#7570b3') #7E1E9C
plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Test Data - Logistic Regression')

plt.savefig('ROC_Test_Split.pdf')

In [ ]:
# Test All Together
plt.figure(figsize = (5,5))

fpr, tpr, thresholds = roc_curve(y_test, rf_best_model.predict_proba(X_test)[:,1])
auc = round(roc_auc_score(y_test, rf_best_model.predict(X_test)), 2)
plt.plot(fpr,tpr,label='Random Forest, AUC='+str(auc), color='#1b9e77')


fpr, tpr, thresholds = roc_curve(y_test, xgb_best_model.predict_proba(X_test)[:,1])
auc = round(roc_auc_score(y_test, xgb_best_model.predict(X_test)),2)
plt.plot(fpr,tpr,label='XGB Classifier, AUC='+str(auc), color ='#d95f02')


fpr, tpr, thresholds = roc_curve(y_test, lr.predict_proba(X_test)[:,1])
auc = round(roc_auc_score(y_test, lr.predict(X_test)),2)
plt.plot(fpr,tpr,label='Logistics Regression, AUC='+str(auc), color='#7570b3') #7E1E9C

plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Test Data')

plt.savefig('ROC_Test_All.pdf')

### Confusion Matrix

In [ ]:
#### TRAIN 
import seaborn as sns

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
fig.tight_layout() # Or equivalently,  "plt.tight_layout()"


#### Random Forest
plt.subplot(1,3,1)
cfm = confusion_matrix(y_train_res, rf_best_model.predict(X_train_res))

class_names=[0,1] # name  of classes
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cfm), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Train Confusion matrix - Random Forest', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

# Text(0.5,257.44,'Predicted label');

#### XGB
plt.subplot(1,3,2)
cfm1 = confusion_matrix(y_train_res, xgb_best_model.predict(X_train_res))

class_names=[0,1] # name  of classes
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cfm1), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Train Confusion matrix - XGB Classifier', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#Text(0.5,257.44,'Predicted label');

#### Logistic Regression
plt.subplot(1,3,3)
cfm2 = confusion_matrix(y_train_res, lr.predict(X_train_res))

class_names=[0,1] # name  of classes
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cfm2), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Train Confusion matrix - Logistic Regression', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#Text(0.5,257.44,'Predicted label')

plt.savefig('ConfusionMatrixTrain.pdf')

In [ ]:
#### VALIDATION
import seaborn as sns

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
fig.tight_layout() # Or equivalently,  "plt.tight_layout()"


#### Random Forest
plt.subplot(1,3,1)
cfm = confusion_matrix(y_val, rf_best_model.predict(X_val))

class_names=[0,1] # name  of classes
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cfm), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Validation Confusion matrix - Random Forest', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

# Text(0.5,257.44,'Predicted label');

#### XGB
plt.subplot(1,3,2)
cfm1 = confusion_matrix(y_val, xgb_best_model.predict(X_val))

class_names=[0,1] # name  of classes
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cfm1), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Validation Confusion matrix - XGB Classifier', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#Text(0.5,257.44,'Predicted label');

#### Logistic Regression
plt.subplot(1,3,3)
cfm2 = confusion_matrix(y_val, lr.predict(X_val))

class_names=[0,1] # name  of classes
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cfm2), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Validation Confusion matrix - Logistic Regression', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#Text(0.5,257.44,'Predicted label')

plt.savefig('ConfusionMatrixVal.pdf')

In [ ]:
#### TEST
import seaborn as sns

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
fig.tight_layout() # Or equivalently,  "plt.tight_layout()"

#### Random Forest
plt.subplot(1,3,1)
cfm = confusion_matrix(y_test, rf_best_model.predict(X_test))

class_names=[0,1] # name  of classes
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cfm), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Test Confusion matrix - Random Forest', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

# Text(0.5,257.44,'Predicted label');

#### XGB
plt.subplot(1,3,2)
cfm1 = confusion_matrix(y_test, xgb_best_model.predict(X_test))

class_names=[0,1] # name  of classes
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cfm1), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Test Confusion matrix - XGB Classifier', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#Text(0.5,257.44,'Predicted label');

#### Logistic Regression
plt.subplot(1,3,3)
cfm2 = confusion_matrix(y_test, lr.predict(X_test))

class_names=[0,1] # name  of classes
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cfm2), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Test Confusion matrix - Logistic Regression', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#Text(0.5,257.44,'Predicted label')

plt.savefig('ConfusionMatrixTest.pdf')

### Compare results from best model parameters. For Tables in Paper Only

In [ ]:
def get_metrics(y,y_hat,y_prob_hat):

    acc_score = round(accuracy_score(y, y_hat),4)
    brier_score = round(brier_score_loss(y,y_prob_hat[:,1], pos_label=1),4)
    log_loss_score = round(log_loss(y,y_prob_hat, labels=[0,1]),4)    
    fpr, tpr, thresholds = metrics.roc_curve(y,y_prob_hat[:,1])
    AUC = metrics.auc(fpr, tpr)     
    
    return acc_score,brier_score,log_loss_score,AUC 

def get_results_from_tunning_RF_LR(tuned_model):
        
    # Train data
    y_train_pred = tuned_model.predict(X_train_res)
    y_train_probs_uncallibrated = tuned_model.predict_proba(X_train_res)

    train_accuracy_score, train_brier_score, train_log_loss_score, train_AUC = get_metrics(y_train_res,y_train_pred,y_train_probs_uncallibrated)
    

    # Validation
    y_val_pred = tuned_model.predict(X_val)
    y_val_probs_uncallibrated = tuned_model.predict_proba(X_val)
    
    val_accuracy_score,val_brier_score,val_log_loss_score,val_AUC = get_metrics(y_val,y_val_pred,y_val_probs_uncallibrated)
    
    # Test data
    y_test_pred = tuned_model.predict(X_test)
    y_test_probs_uncallibrated = tuned_model.predict_proba(X_test)
    
    test_accuracy_score,test_brier_score,test_log_loss_score,test_AUC = get_metrics(y_test, y_test_pred, y_test_probs_uncallibrated)

  
    # Callibrated  - test data
    
    #sigmoid
    callibrated_sigmoid = CalibratedClassifierCV(tuned_model, method="sigmoid", cv="prefit") # isotonic and sigmoid is tested - isotonic performed better
    callibrated_sigmoid.fit(X_train_res, y_train_res)
    
    callibrated_sigmoid_probs = callibrated_sigmoid.predict_proba(X_test)
    y_test_pred_callibrated_sigmoid = callibrated_sigmoid.predict(X_test)
        
    sigmoid_accuracy_score,brier_score_sigmoid,callibrated_log_loss_sigmoid,sigmoid_AUC = get_metrics(y_test, y_test_pred_callibrated_sigmoid, callibrated_sigmoid_probs)

    # isotonic 
    callibrated_isotonic = CalibratedClassifierCV(tuned_model, method="isotonic", cv="prefit")
    callibrated_isotonic.fit(X_train_res, y_train_res)
    
    callibrated_isotonic_probs = callibrated_isotonic.predict_proba(X_test)
    y_test_pred_callibrated_isotonic = callibrated_isotonic.predict(X_test)
    
    isotonic_accuracy_score,brier_score_isotonic,callibrated_log_loss_isotonic,isotonic_AUC = get_metrics(y_test, y_test_pred_callibrated_isotonic, callibrated_isotonic_probs)
    
    
    results = [train_accuracy_score,
               train_brier_score,
               train_log_loss_score,
               train_AUC,
               
               val_accuracy_score,
               val_brier_score,
               val_log_loss_score,
               val_AUC,
               
               test_accuracy_score,
               test_brier_score,
               test_log_loss_score,
               test_AUC,
               
               sigmoid_accuracy_score,
               brier_score_sigmoid,
               callibrated_log_loss_sigmoid,
               sigmoid_AUC,
               
               isotonic_accuracy_score,
               brier_score_isotonic,
               callibrated_log_loss_isotonic,
               isotonic_AUC
    
    
    ]    
    return results, callibrated_isotonic


def get_results_from_tunning_XGB(tuned_model):
        
    # Train data
    y_train_pred = tuned_model.predict(train_x) 
    y_train_probs_uncallibrated = tuned_model.predict_proba(train_x)

    train_accuracy_score, train_brier_score, train_log_loss_score, train_AUC = get_metrics(train_y,y_train_pred,y_train_probs_uncallibrated)
    

    # Validation
    y_val_pred = tuned_model.predict(val_x) 
    y_val_probs_uncallibrated = tuned_model.predict_proba(val_x)
    
    val_accuracy_score,val_brier_score,val_log_loss_score,val_AUC = get_metrics(val_y,y_val_pred,y_val_probs_uncallibrated)
    
    # Test data
    y_test_pred = tuned_model.predict(test_x) 
    y_test_probs_uncallibrated = tuned_model.predict_proba(test_x)

    test_accuracy_score,test_brier_score,test_log_loss_score,test_AUC = get_metrics(test_y, y_test_pred, y_test_probs_uncallibrated)

  
    # Callibrated  - test data
    
    #sigmoid
    callibrated_sigmoid = CalibratedClassifierCV(tuned_model, method="sigmoid", cv="prefit") 
    callibrated_sigmoid.fit(train_x,train_y)
    
    callibrated_sigmoid_probs = callibrated_sigmoid.predict_proba(test_x)
    y_test_pred_callibrated_sigmoid = callibrated_sigmoid.predict(test_x)
        
    sigmoid_accuracy_score,brier_score_sigmoid,callibrated_log_loss_sigmoid,sigmoid_AUC = get_metrics(test_y, y_test_pred_callibrated_sigmoid, callibrated_sigmoid_probs)

    # isotonic 
    callibrated_isotonic = CalibratedClassifierCV(tuned_model, method="isotonic", cv="prefit")
    callibrated_isotonic.fit(train_x,train_y)
    
    callibrated_isotonic_probs = callibrated_isotonic.predict_proba(test_x)
    y_test_pred_callibrated_isotonic = callibrated_isotonic.predict(test_x)
    
    isotonic_accuracy_score,brier_score_isotonic,callibrated_log_loss_isotonic,isotonic_AUC = get_metrics(test_y, y_test_pred_callibrated_isotonic, callibrated_isotonic_probs)
    
    
    results = [train_accuracy_score,
               train_brier_score,
               train_log_loss_score,
               train_AUC,
               
               val_accuracy_score,
               val_brier_score,
               val_log_loss_score,
               val_AUC,
               
               test_accuracy_score,
               test_brier_score,
               test_log_loss_score,
               test_AUC,
               
               sigmoid_accuracy_score,
               brier_score_sigmoid,
               callibrated_log_loss_sigmoid,
               sigmoid_AUC,
               
               isotonic_accuracy_score,
               brier_score_isotonic,
               callibrated_log_loss_isotonic,
               isotonic_AUC
    
    
    ]    
    return results,callibrated_isotonic

In [ ]:
results_rf,callibrated_rf = get_results_from_tunning_RF_LR(rf_best_model) 
results_rf


In [ ]:
results_xgb,callibrated_xgb = get_results_from_tunning_XGB(xgb_best_model) 
results_xgb

In [ ]:
results_lr,callibrated_lr = get_results_from_tunning_RF_LR(lr) 
results_lr

In [ ]:
#rf model 
if MODEL_TYPE == 'XGB':
    results,callibrated_ml = get_results_from_tunning_XGB(xgb_best_model) 
if MODEL_TYPE == 'RF':
    results,callibrated_ml = get_results_from_tunning_RF_LR(rf_best_model) 
if MODEL_TYPE == 'LR':
    results,callibrated_ml = get_results_from_tunning_RF_LR(lr) 
# if MODEL_TYPE == 'STM':
    # results,callibrated_ml = get_results_from_tunning_RF_LR(result)    
    

In [ ]:
callibrated_ml

# Permutation Feature Importance

In [ ]:
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# RF Feature importance
print(pd.DataFrame({'columns':X_train.columns, 'feature_importance':rf_best_model.feature_importances_}).sort_values(by='feature_importance', ascending=False))


In [ ]:
# XGB Feature Importance
print(pd.DataFrame({'columns':X_train.columns, 'feature_importance':xgb_best_model.feature_importances_}).sort_values(by='feature_importance', ascending=False))


In [ ]:
# Test Data Feature Importance
print(pd.DataFrame({'columns':X_train.columns, 'feature_importance':xgb_best_model.feature_importances_}).sort_values(by='feature_importance', ascending=False))


In [ ]:
# baseline train before sampling
if MODEL_TYPE == 'RF':
    print(pd.DataFrame({'columns':X_train.columns, 'feature_importance':rf_best_model.feature_importances_}).sort_values(by='feature_importance', ascending=False))
if MODEL_TYPE == 'XGB':
    print(pd.DataFrame({'columns':X_train.columns, 'feature_importance':xgb_best_model.feature_importances_}).sort_values(by='feature_importance', ascending=False))
    

In [ ]:
# Test Data
if MODEL_TYPE == 'RF':
    perm = PermutationImportance(callibrated_ml, random_state=1).fit(X_test, y_test)
if MODEL_TYPE == 'XGB':
    perm = PermutationImportance(callibrated_ml, random_state=1).fit(test_x, test_y)
    

In [ ]:
eli5.show_weights(perm,feature_names = features)

In [ ]:
feature_for_plot = ['Departure: Hour of Day-Sin',
 'Departure: Hour of Day-Cos',
 'Departure: Hour of Day',
 'Hook Trailer Time',
 'Drop Trailer Time',
 'Transit Time',
 'Miles',
 'Check-in Time Window',
 'Total Block Time',
 'Number of Available Blocks',
 'Departure: Day of Week',
 'Departure: Week of Year',
 'Origin Zip',
 'Destination Zip',
 'Time to Departure']


## Random Forest - Permutation Feature Importance - Train

In [ ]:
# Train Data - Random Foreast
# perm_train_rf = PermutationImportance(rf_best_model, random_state=1).fit(X_train_res, y_train_res)

df_results = pd.DataFrame(data = perm_train_rf.results_, columns= feature_for_plot)
feat_imps = df_results.mean().sort_values(ascending=False)
df_results = df_results[feat_imps.index]
fig = px.box(df_results.melt(), x='variable', y='value', orientation='v', width=700, height=400, 
            title = 'Permutation Feature Importance - Train Data - Random Forest')
fig.add_trace(go.Scatter(x=feat_imps.index, y=feat_imps.values, mode='markers', marker=dict(color='red'), name = 'Mean'))
fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="green",
    plot_bgcolor='white'
    )

fig.update_xaxes(showline=True, linewidth=1, linecolor='black',title_font_family="Helvetica")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.85
))

# plotly.offline.plot(fig, filename='fi_test.html')

fig

In [ ]:
eli5.show_weights(perm_train_rf,feature_names = feature_for_plot)

## XGB - Permutation Feature Importance - Train

In [ ]:
# Train Data - XGB
perm_train_xgb = PermutationImportance(xgb_best_model, random_state=1).fit(train_x, train_y)

df_results = pd.DataFrame(data=perm_train_xgb.results_, columns= feature_for_plot)
feat_imps = df_results.mean().sort_values(ascending=False)
df_results = df_results[feat_imps.index]
fig = px.box(df_results.melt(), x='variable', y='value', orientation='v', width=700, height=400, 
            title = 'Permutation Feature Importance - Train Data -XGB')
fig.add_trace(go.Scatter(x=feat_imps.index, y=feat_imps.values, mode='markers', marker=dict(color='red'), name = 'Mean'))
fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="green",
    plot_bgcolor='white'
    )


fig.update_xaxes(showline=True, linewidth=1, linecolor='black',title_font_family="Helvetica")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.85
))

# plotly.offline.plot(fig, filename='fi_test.html')

fig

In [ ]:
eli5.show_weights(perm_train_xgb,feature_names = feature_for_plot)

In [ ]:
perm_train_xgb.results_

## LR - Permutation Feature Importance - Train

In [ ]:
# Train Data - Logistic Regression
perm_train_lr = PermutationImportance(lr, random_state=1).fit(train_x, train_y)

df_results = pd.DataFrame(data=perm_train_lr.results_, columns= feature_for_plot)

feat_imps = df_results.mean().sort_values(ascending=False)
df_results = df_results[feat_imps.index]
fig = px.box(df_results.melt(), x='variable', y='value', orientation='v', width=700, height=400, 
            title = 'Permutation Feature Importance - Train Data - LR')
fig.add_trace(go.Scatter(x=feat_imps.index, y=feat_imps.values, mode='markers', marker=dict(color='red'), name = 'Mean'))
fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="green",
    plot_bgcolor='white'
    )


fig.update_xaxes(showline=True, linewidth=1, linecolor='black',title_font_family="Helvetica")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.85
))

# plotly.offline.plot(fig, filename='fi_test.html')

fig

In [ ]:
eli5.show_weights(perm_train_lr,feature_names = feature_for_plot)

# Permutation Importance on Test Data

## Random Forest - Permutation Feature Importance - Test

In [ ]:
# Train Data - Random Foreast
perm_test_rf = PermutationImportance(rf_best_model, random_state=1).fit(X_test, y_test)

df_results = pd.DataFrame(data = perm_test_rf.results_, columns= feature_for_plot)
feat_imps = df_results.mean().sort_values(ascending=False)
df_results = df_results[feat_imps.index]
fig = px.box(df_results.melt(), x='variable', y='value', orientation='v', width=700, height=400, 
            title = 'Permutation Feature Importance - Test Data - Random Forest')
fig.add_trace(go.Scatter(x=feat_imps.index, y=feat_imps.values, mode='markers', marker=dict(color='red'), name = 'Mean'))
fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="green",
    plot_bgcolor='white'
    )

fig.update_xaxes(showline=True, linewidth=1, linecolor='black',title_font_family="Helvetica")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.85
))

# plotly.offline.plot(fig, filename='fi_test.html')

fig

In [ ]:
eli5.show_weights(perm_test_rf,feature_names = feature_for_plot)

## XGB - Permutation Feature Importance - Test

In [ ]:
# Test data using XGB
perm_test_xgb = PermutationImportance(xgb_best_model, random_state=1).fit(test_x, test_y)

df_results = pd.DataFrame(data=perm_test_xgb.results_, columns= feature_for_plot)

feat_imps = df_results.mean().sort_values(ascending=False)
df_results = df_results[feat_imps.index]
fig = px.box(df_results.melt(), x='variable', y='value', orientation='v', width=700, height=400, 
            title = 'Permutation Feature Importance - Test Data - XGB')
fig.add_trace(go.Scatter(x=feat_imps.index, y=feat_imps.values, mode='markers', marker=dict(color='red'), name = 'Mean'))

fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="green",
    plot_bgcolor='white'
    )
fig.update_xaxes(showline=True, linewidth=1, linecolor='black',title_font_family="Helvetica")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.85
))

# plotly.offline.plot(fig, filename='fi_test.html')

fig

In [ ]:
eli5.show_weights(perm_test_xgb,feature_names = feature_for_plot)

## Logistic Regression - Permutation Feature Importance - Test

In [ ]:
# Test data using XGB
perm_test_lr = PermutationImportance(lr, random_state=1).fit(test_x, test_y)

df_results = pd.DataFrame(data=perm_test_lr.results_, columns= feature_for_plot)

feat_imps = df_results.mean().sort_values(ascending=False)
df_results = df_results[feat_imps.index]
fig = px.box(df_results.melt(), x='variable', y='value', orientation='v', width=700, height=400, 
            title = 'Permutation Feature Importance - Test Data - Logistic Regression')
fig.add_trace(go.Scatter(x=feat_imps.index, y=feat_imps.values, mode='markers', marker=dict(color='red'), name = 'Mean'))

fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="green",
    plot_bgcolor='white'
    )
fig.update_xaxes(showline=True, linewidth=1, linecolor='black',title_font_family="Helvetica")
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.85
))

# plotly.offline.plot(fig, filename='fi_test.html')

fig

In [ ]:
eli5.show_weights(perm_test_lr,feature_names = feature_for_plot)

## Compare model performance on Test Data

In [ ]:
X_test_load_related.head()

In [ ]:
X_test.head()

In [ ]:
# bring back load ID
X_test_full = pd.merge(X_test_load_related, X_test, left_index=True, right_index=True)
X_test_full.head()

## Using XGB

In [ ]:
# using xgb as the best model for prediction
callibrated_xgb = xgb_best_model

In [ ]:
callibrated_xgb_probs = callibrated_xgb.predict_proba(X_test)
y_test_pred_callibrated_xgb = callibrated_xgb.predict(X_test)

# 0 is planned, 1 is unplanned
y_test_probs_callibrated_df = pd.DataFrame(data=callibrated_xgb_probs, 
                                           index=X_test_full.index, columns=["planned", "unplanned"])

full_df = X_test_full.copy()
full_df['is_eventually_unplanned_actual'] = y_test.copy()
full_df['is_eventually_unplanned_pred'] = y_test_pred_callibrated_xgb.copy()
full_df['probability of unplanned'] = y_test_probs_callibrated_df['unplanned'].copy()
full_df['probability of planned'] = y_test_probs_callibrated_df['planned'].copy()


In [ ]:
pd.Series(full_df.is_eventually_unplanned_actual.value_counts())

In [ ]:
cols = ['load_id','probability of planned','probability of unplanned']

In [ ]:
hist_unplanned = full_df.hist(column='probability of unplanned',by=['is_eventually_unplanned_actual'],bins=10, grid=True, 
                              figsize=(10,4),color='#d95f02',ylabelsize = 12, xlabelsize = 12)

for x in hist_unplanned:
    if x == hist_unplanned[0]:
        x.set_title("Acutally Planned",fontsize = 12)
    else: 
        x.set_title("Acutally Unplanned",fontsize = 12)
    x.set_xlabel("Probability of being unplanned - XGB", fontsize = 12) 
    x.set_ylabel("Number of Truckloads", fontsize = 12)

plt.savefig('Unplan_xgb.pdf')


In [ ]:
hist_planned = full_df.hist(column='probability of planned',by=['is_eventually_unplanned_actual'],bins=10, grid=True, 
                              figsize=(10,4),color='#d95f02',ylabelsize = 12, xlabelsize = 12)

for x in hist_planned:
    if x == hist_planned[1]:
        x.set_title("Acutally Unplanned",fontsize = 12)
    else: 
        x.set_title("Acutally Planned",fontsize = 12)
    x.set_xlabel("Probability of being planned - XGB", fontsize = 12) 
    x.set_ylabel("Number of Truckloads", fontsize = 12)

plt.savefig('Plan_xgb.pdf')

In [ ]:
matrix_xgb = classification_report(y_test,y_test_pred_callibrated_xgb,labels=[1,0])
print(matrix_xgb)

## Using Logistic Regression

In [ ]:
# using xgb as the best model for prediction
callibrated_lr = lr

In [ ]:
callibrated_lr_probs = callibrated_lr.predict_proba(X_test)
y_test_pred_callibrated_lr = callibrated_lr.predict(X_test)

# 0 is planned, 1 is unplanned
y_test_probs_callibrated_df = pd.DataFrame(data=callibrated_lr_probs, 
                                           index=X_test_full.index, columns=["planned", "unplanned"])

full_df = X_test_full.copy()
full_df['is_eventually_unplanned_actual'] = y_test.copy()
full_df['is_eventually_unplanned_pred'] = y_test_pred_callibrated_lr.copy()
full_df['probability of unplanned'] = y_test_probs_callibrated_df['unplanned'].copy()
full_df['probability of planned'] = y_test_probs_callibrated_df['planned'].copy()


In [ ]:
pd.Series(full_df.is_eventually_unplanned_actual.value_counts())

In [ ]:
cols = ['load_id','probability of planned','probability of unplanned']

In [ ]:
hist_unplanned = full_df.hist(column='probability of unplanned',by=['is_eventually_unplanned_actual'],bins=10, grid=True, 
                              figsize=(10,4),color='#7570b3',ylabelsize = 12, xlabelsize = 12)

for x in hist_unplanned:
    if x == hist_unplanned[0]:
        x.set_title("Acutally Planned",fontsize = 12)
    else: 
        x.set_title("Acutally Unplanned",fontsize = 12)
    x.set_xlabel("Probability of being unplanned - Logistic Regression", fontsize = 12) 
    x.set_ylabel("Number of Truckloads", fontsize = 12)

plt.savefig('Unplan_lr.pdf')

In [ ]:
hist_planned = full_df.hist(column='probability of planned',by=['is_eventually_unplanned_actual'],bins=10, grid=True, 
                              figsize=(10,4),color='#7570b3',ylabelsize = 12, xlabelsize = 12)

for x in hist_planned:
    if x == hist_planned[1]:
        x.set_title("Acutally Unplanned",fontsize = 12)
    else: 
        x.set_title("Acutally Planned",fontsize = 12)
    x.set_xlabel("Probability of being planned - Logistic Regression", fontsize = 12) 
    x.set_ylabel("Number of Truckloads", fontsize = 12)

plt.savefig('Plan_lr.pdf')

In [ ]:
matrix_lr = classification_report(y_test,y_test_pred_callibrated_lr,labels=[1,0])
print(matrix_lr)

In [ ]:
import os
os.getcwd()

## Using Random Forest

In [ ]:
# using xgb as the best model for prediction
callibrated_rf = rf_best_model

In [ ]:
callibrated_rf_probs = callibrated_rf.predict_proba(X_test)
y_test_pred_callibrated_rf = callibrated_rf.predict(X_test)

# 0 is planned, 1 is unplanned
y_test_probs_callibrated_df = pd.DataFrame(data = callibrated_rf_probs, 
                                           index = X_test_full.index, columns=["planned", "unplanned"])

full_df = X_test_full.copy()
full_df['is_eventually_unplanned_actual'] = y_test.copy()
full_df['is_eventually_unplanned_pred'] = y_test_pred_callibrated_rf.copy()
full_df['probability of unplanned'] = y_test_probs_callibrated_df['unplanned'].copy()
full_df['probability of planned'] = y_test_probs_callibrated_df['planned'].copy()

In [ ]:
pd.Series(full_df.is_eventually_unplanned_actual.value_counts())

In [ ]:
cols = ['load_id','probability of planned','probability of unplanned']

In [ ]:
hist_unplanned = full_df.hist(column='probability of unplanned',by=['is_eventually_unplanned_actual'],bins=10, grid=True, 
                              figsize=(10,4),color='#1c9c74',ylabelsize = 12, xlabelsize = 12)

for x in hist_unplanned:
    if x == hist_unplanned[0]:
        x.set_title("Acutally Planned",fontsize = 12)
    else: 
        x.set_title("Acutally Unplanned",fontsize = 12)
    x.set_xlabel("Probability of being unplanned - Random Forest", fontsize = 12) 
    x.set_ylabel("Number of Truckloads", fontsize = 12)

plt.savefig('Unplan_rf.pdf')

In [ ]:
hist_planned = full_df.hist(column='probability of planned',by=['is_eventually_unplanned_actual'],bins=10, grid=True, 
                              figsize=(10,4),color='#1c9c74',ylabelsize = 12, xlabelsize = 12)

for x in hist_planned:
    if x == hist_planned[1]:
        x.set_title("Acutally Unplanned",fontsize = 12)
    else: 
        x.set_title("Acutally Planned",fontsize = 12)
    x.set_xlabel("Probability of being planned - Random Forest", fontsize = 12) 
    x.set_ylabel("Number of Truckloads", fontsize = 12)

plt.savefig('Plan_rf.pdf')

In [ ]:
matrix_rf = classification_report(y_test,y_test_pred_callibrated_rf,labels=[1,0])
print(matrix_rf)

# Partial Dependence Plot
### Following Cell Takes about 30 minutes 

In [ ]:
features = ['sin_departure_time_hour_of_day',
            'cos_departure_time_hour_of_day',
            'departure_hour_of_day',
            'hook_trailer_min',
            'drop_trailer_min',
            'average_transit_hour',
            'miles',
            'checkin_time_windows_at_origin',
            'total_block_minutes',
            'num_feasible_blocks',
            'departure_day_of_week',
            'departure_week_of_year',
            'origin_zip3',
            'dest_zip3',
            'lead_time_to_departure'
           ]

#plot_partial_dependence(callibrated_ml, X_train_res, features, n_jobs=3, grid_resolution=20)

rf_disp = PartialDependenceDisplay.from_estimator(rf_best_model, X_train_res, features, n_jobs=3, grid_resolution=20)
xgb_disp = PartialDependenceDisplay.from_estimator(xgb_best_model, X_train_res, features, n_jobs=3, grid_resolution=20)
lr_disp = PartialDependenceDisplay.from_estimator(lr, X_train_res, features, n_jobs = 3,grid_resolution=20)


In [ ]:
# plot based on results from previous step
rf_disp.plot(line_kw={"label": "Random Forest","color": "#1c9c74"})
xgb_disp.plot(line_kw={"label": "XGB", "color": "#dc5c04"}, ax=rf_disp.axes_)
lr_disp.plot(line_kw={"label": "Logistic Regression", "color": "#7474b4"}, ax=rf_disp.axes_)

lr_disp.figure_.set_size_inches(12, 18)
lr_disp.axes_[0, 0].legend()
lr_disp.axes_[0, 1].legend()

plt.savefig('PartialDependece.pdf')

# Last Step - Predicting Next Week's Data


In [ ]:
df2020 = pd.read_csv(data_dir + 'train_2020.csv',sep='\t', low_memory = False)
len(df2020)

In [ ]:
df2020['departure_hour_of_day'] = pd.to_datetime(df2020['first_checkin_time_utc']).dt.hour
df2020['departure_day_of_week'] = pd.to_datetime(df2020['first_checkin_time_utc']).dt.dayofweek
df2020['departure_week_of_year'] = pd.to_datetime(df2020['first_checkin_time_utc']).dt.weekofyear
df2020['departure_day_of_month'] = pd.to_datetime(df2020['first_checkin_time_utc']).dt.day
df2020['departure_month'] = pd.to_datetime(df2020['first_checkin_time_utc']).dt.month

df2020['sin_departure_time_hour_of_day'] = np.sin((2*np.pi*df2020['departure_hour_of_day'])/24)
df2020['cos_departure_time_hour_of_day'] = np.cos((2*np.pi*df2020['departure_hour_of_day'])/24)

df2020['creation_week_of_year'] = pd.to_datetime(df2020['creation_date']).dt.weekofyear


In [ ]:
w50_all = df2020[(df2020.week == 50)]
w50_all.shape

In [ ]:
w50_team = pd.read_csv(data_dir + 'w50.txt', sep = "\t")
w50_team.shape

In [ ]:
x = set(df2020.load_id.unique())
y = set(w50_team.Load_id.unique())

print(len(y))

In [ ]:
len(x.intersection(y)) # 1058 was kept to be planned. The remaining sent to RLB.

In [ ]:
df2020['flag'] = np.where(df2020['load_id'].isin (y),1,0)

In [ ]:
df_50 = df2020[df2020.flag == 1]
df_50.shape

In [ ]:
(df_50['is_eventually_unplanned'].value_counts())[1]/len(df_50['is_eventually_unplanned'])

In [ ]:
df_50 = df_50[~df_50['origin_zip3'].isnull()]
len(df_50)

In [ ]:
df_50['origin_zip3'] = df_50['origin_zip3'].astype(int)
df_50['dest_zip3'] = df_50['dest_zip3'].astype(int)

In [ ]:
callibrated_ml = xgb_best_model

In [ ]:
X_test_50 = df_50[features].copy()

assert(len(X_test_50[X_test_50['lead_time_to_departure'] < 0]) == 0)
X_test_50_load_related = df_50[load_related_features].copy()

X_test_full_50 = pd.merge(X_test_50_load_related, X_test_50, left_index=True, right_index=True)

callibrated_ml_model_probs = callibrated_ml.predict_proba(X_test_50)

y_test_probs_callibrated_df = pd.DataFrame(data=callibrated_ml_model_probs, index=X_test_full_50.index, 
                                           columns=["planned", "unplanned"])

full_df = X_test_full_50.copy()
full_df['probability of unplanned'] = y_test_probs_callibrated_df['unplanned'].copy()
full_df['probability of planned'] = y_test_probs_callibrated_df['planned'].copy()    


In [ ]:
full_df.head()

In [ ]:
CUT_OFF_POINT = 0.5
loads_sent_to_rlb = full_df[full_df['probability of unplanned'] >= CUT_OFF_POINT]
print(len(loads_to_sent_to_rlb))

In [ ]:
loads_held = df_50[~df_50['load_id'].isin(loads_to_sent_to_rlb['load_id'])]
print(len(loads_held))

In [ ]:
loads_held = full_df[full_df['probability of unplanned'] < CUT_OFF_POINT]
print(len(loads_held))

In [ ]:
loads_held['probability of unplanned'].hist()

In [ ]:
percent_loads_sent_to_rlb = round(len(loads_sent_to_rlb)/ len(full_df) *100,2)
percent_loads_sent_to_rlb
print('Percentage of loads sent to rlb is:',percent_loads_sent_to_rlb)

In [ ]:
cut_off_results = defaultdict(list)

for CUT_OFF in CUT_OFF_POINTS: 
    print('CUT OFF : %s' % CUT_OFF)
    CUT_OFF_POINT = CUT_OFF/100
 
    cols = ['load_id','probability of planned','probability of unplanned']
    team_loads_current_week = full_df[cols].copy()

    loads_sent_to_rlb = team_loads_current_week[team_loads_current_week['probability of unplanned'] >= CUT_OFF_POINT]
    print(len(loads_sent_to_rlb))
    
    loads_held = team_loads_current_week[team_loads_current_week['probability of unplanned'] < CUT_OFF_POINT]
    print(len(loads_held))
    

    cols = ['load_id','probability of unplanned']
    loads_held = loads_held[cols].copy()   

    
    percent_loads_sent_to_rlb = round(len(loads_sent_to_rlb) / len(team_loads_current_week) *100,2)
    print('---')
    print(loads_sent_to_rlb)
    print(percent_loads_sent_to_rlb)
    
    cut_off_results['%s'% CUT_OFF] = [len(loads_sent_to_rlb),loads_sent_to_rlb,percent_loads_sent_to_rlb]
    
   

In [ ]:
result = pd.DataFrame(data=cut_off_results)
print(result)

In [ ]:
full_df.columns

## Calculate Actual Performance to decide which cut off to use

In [ ]:
p50_held = full_df[full_df['probability of unplanned'] <=0.5]
p50_held.shape

In [ ]:
p60_held = full_df[full_df['probability of unplanned'] <=0.6]
p60_held.shape

In [ ]:
p70_held = full_df[full_df['probability of unplanned'] <=0.7]
p70_held.shape

In [ ]:
p50_actual = df_50[df_50['load_id'].isin (p50_held['load_id'])]
print((p50_actual['is_eventually_unplanned'].value_counts()))
(p50_actual['is_eventually_unplanned'].value_counts()[0])/len(p50_actual)

In [ ]:
p60_actual = df_50[df_50['load_id'].isin (p60_held['load_id'])]
print(p60_actual['is_eventually_unplanned'].value_counts())
(p60_actual['is_eventually_unplanned'].value_counts()[0])/len(p60_actual)

In [ ]:
p70_actual = df_50[df_50['load_id'].isin (p70_held['load_id'])]
print(p70_actual['is_eventually_unplanned'].value_counts())
(p70_actual['is_eventually_unplanned'].value_counts()[0])/len(p70_actual)

### ROC for Unplanned Based on Probability Cut Off

In [ ]:
y_50 = df_50['is_eventually_unplanned']
y_50

In [ ]:
full_df.info()

In [ ]:
full_df['Predict_50'] = np.where(full_df['probability of unplanned'] >= 0.5, 1,0)
full_df['Predict_60'] = np.where(full_df['probability of unplanned'] >= 0.6, 1,0)
full_df['Predict_70'] = np.where(full_df['probability of unplanned'] >= 0.7, 1,0)

In [ ]:
full_df.info()

In [ ]:
full_df.head()

In [ ]:
plt.figure(figsize = (5,5))

fpr, tpr, thresholds = roc_curve(y_50, xgb_best_model.predict_proba(X_test_50)[:,1])
auc = round(roc_auc_score(y_50, xgb_best_model.predict(X_test_50)),2)
plt.plot(fpr,tpr,label='Cut Off, AUC='+str(auc), color='#666666')

plt.plot([0, 1], [0, 1],'r--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Future Week Model Performance - Post Hoc Analysis')

plt.savefig('ROC_W50_XGB.pdf')

In [ ]:
roc_auc_score(y_50, np.array(full_df['Predict_50']))

In [ ]:
roc_auc_score(y_50, np.array(full_df['Predict_60']))

In [ ]:
roc_auc_score(y_50, np.array(full_df['Predict_70']))

In [ ]:
# Test All Together
plt.figure(figsize = (5,5))

fpr, tpr, thresholds = roc_curve(y_50, rf_best_model.predict_proba(X_test_50)[:,1])
auc = round(roc_auc_score(y_50, np.array(full_df['Predict_50'])),2)
plt.plot(fpr,tpr,label='Cut Off at 50%, AUC='+str(auc), color='#1b9e77')


fpr, tpr, thresholds = roc_curve(y_50, xgb_best_model.predict_proba(X_test_50)[:,1])
auc = round(roc_auc_score(y_50, np.array(full_df['Predict_60'])),2)
plt.plot(fpr,tpr,label='Cut Off at 60%, AUC='+str(auc), color ='#d95f02')

fpr, tpr, thresholds = roc_curve(y_50, lr.predict_proba(X_test_50)[:,1])
auc = round(roc_auc_score(y_50, np.array(full_df['Predict_70'])),2)
plt.plot(fpr,tpr,label='Cut Off at 70%, AUC='+str(auc), color='#7570b3') #7E1E9C

plt.plot([0, 1], [0, 1],'k--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Test Data')

plt.savefig('ROC_W50_All.pdf')

In [ ]:
xgb_best_model.predict_proba(X_test_50)

In [ ]:
# Test Seperate

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 3.4))
# fig.tight_layout() # Or equivalently,  "plt.tight_layout()"

plt.subplot(1,3,1)
fpr, tpr, thresholds = roc_curve(y_50, xgb_best_model.predict_proba(X_test_50)[:,1])
auc = round(roc_auc_score(y_50, np.array(full_df['Predict_50'])),2)
plt.plot(fpr,tpr,label='Cut Off at 50%, AUC='+str(auc), color='#666666')
plt.plot([0, 1], [0, 1],'r--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Cut off at 50%')

plt.subplot(1,3,2)
fpr, tpr, thresholds = roc_curve(y_50, xgb_best_model.predict_proba(X_test_50)[:,1])
auc = round(roc_auc_score(y_50, np.array(full_df['Predict_60'])),2)
plt.plot(fpr,tpr,label='Cut Off at 60%, AUC='+str(auc), color ='#a6761d')
plt.plot([0, 1], [0, 1],'r--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Cut off at 60%')

plt.subplot(1,3,3)
fpr, tpr, thresholds = roc_curve(y_50, xgb_best_model.predict_proba(X_test_50)[:,1])
auc = round(roc_auc_score(y_50, np.array(full_df['Predict_70'])),2)
plt.plot(fpr,tpr,label='Cut Off at 70%, AUC='+str(auc), color='#e6ab02') #7E1E9C
plt.plot([0, 1], [0, 1],'r--')
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Cut off at 70%')

plt.savefig('ROC_W50_Split.pdf')

## Using Predicted Value Rather than Probability

In [ ]:
X_test_50 = df_50[features].copy()

assert(len(X_test_50[X_test_50['lead_time_to_departure'] < 0]) == 0)
X_test_50_load_related = df_50[load_related_features].copy()

X_test_full_50 = pd.merge(X_test_50_load_related, X_test_50, left_index=True, right_index=True)

callibrated_ml_model_probs = callibrated_ml.predict(X_test_50)

y_test_probs_callibrated_df = pd.DataFrame(data=callibrated_ml_model_probs, index=X_test_full_50.index, 
                                           columns=["PredictedValue"])

full_df = X_test_full_50.copy()
full_df['PredictedValue'] = y_test_probs_callibrated_df['PredictedValue'].copy()


In [ ]:
full_df.head()

In [ ]:
sent =full_df[full_df['PredictedValue'] == 1] 

In [ ]:
held =full_df[full_df['PredictedValue'] == 0]
held.shape

In [ ]:
held_actual = df_50[df_50['load_id'].isin (held['load_id'])]

In [ ]:
(held_actual['is_eventually_unplanned'].value_counts()[1])/len(held_actual)